##기본 설정은 TPU로 해야 합니다. TPU를 활성화한 후 진행하세요.

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.2 MB/s eta 0:00:00


In [ ]:
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=328f8ce55b8640797aa30f0603084e4ae940cde5fdc3dae13941556e57c2438e
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [ ]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from transformers import shape_list, BertTokenizer, TFBertModel
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences
from seqeval.metrics import f1_score, classification_report
import tensorflow as tf
import urllib.request

# 1. 데이터 로드

##Klue 데이터, 라벨 데이터 로드

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/ukairia777/tensorflow-nlp-tutorial/main/18.%20Fine-tuning%20BERT%20(Cls%2C%20NER%2C%20NLI)/dataset/ner_train_data.csv", filename="ner_train_data.csv")
urllib.request.urlretrieve("https://raw.githubusercontent.com/ukairia777/tensorflow-nlp-tutorial/main/18.%20Fine-tuning%20BERT%20(Cls%2C%20NER%2C%20NLI)/dataset/ner_test_data.csv", filename="ner_test_data.csv")
urllib.request.urlretrieve("https://raw.githubusercontent.com/ukairia777/tensorflow-nlp-tutorial/main/18.%20Fine-tuning%20BERT%20(Cls%2C%20NER%2C%20NLI)/dataset/ner_label.txt", filename="ner_label.txt")

('ner_label.txt', <http.client.HTTPMessage at 0x7e9b25ed5cf0>)

##네이버 챌린지 데이터 로드

In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/naver/nlp-challenge/master/missions/ner/data/train/train_data", filename="naver_train_data.csv")

('naver_train_data.csv', <http.client.HTTPMessage at 0x7e9b25ed5c00>)

##네이버 NLP 챌린지 데이터를 klue 데이터셋과 동일하게 수정

In [ ]:
# 데이터 읽기
df = pd.read_csv("naver_train_data.csv", delimiter='\t', names=['Index', 'Word', 'Tag'])

# 문장 번호 만들기
df['Sentence_Num'] = (df['Index'] == 1).cumsum()

# 각 문장에 대해 단어들을 합치기
sentence = df.groupby('Sentence_Num')['Word'].apply(lambda s: ' '.join(s)).reset_index()
sentence.columns = ['Sentence_Num', 'Sentence']

# 각 문장에 대해 태그들을 합치기
tag = df.groupby('Sentence_Num')['Tag'].apply(lambda s: ' '.join(s)).reset_index()
tag.columns = ['Sentence_Num', 'Tag']

# 두 데이터프레임을 합치기
naver_train_df = pd.merge(sentence, tag, on='Sentence_Num')[['Sentence', 'Tag']]

# 결과 저장
naver_train_df.to_csv("new_data.csv", index=False)

In [ ]:
#개체명 아닌 것들 -표기에서 O표기로
naver_train_df['Tag'] = naver_train_df['Tag'].str.replace('-', 'O')
#PER_B를 PER-B로
naver_train_df['Tag'] = naver_train_df['Tag'].str.replace('_', '-')


##Klue, navernlp, 자체구축 데이터셋 합치기

In [ ]:
#자체구축 데이터(상호명, 주소구축, 주소보통명사) 로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/2Moogi/Alice/master/madedata.txt", filename="made_train_data.csv")
made_train_data_df = pd.read_csv("made_train_data.csv")

In [ ]:
# Klue 데이터 -> CSV 파일을 데이터프레임으로 읽어오기
klue_train_df = pd.read_csv('ner_train_data.csv')

In [ ]:
# 행 방향으로 합치기 (위 아래로 합치기)
result = pd.concat([klue_train_df, naver_train_df,made_train_data_df], ignore_index=True)

In [ ]:
# 훈련데이터 result의 모든 ORG 태그를 LOC로 바꾸는 코드
result['Tag'] = result['Tag'].str.replace('ORG-B', 'LOC')
result['Tag'] = result['Tag'].str.replace('ORG-I', 'LOC')
result['Tag'] = result['Tag'].str.replace('LOC-B', 'LOC')
result['Tag'] = result['Tag'].str.replace('LOC-I', 'LOC')

In [ ]:
from pprint import pprint

In [ ]:
df = pd.DataFrame(result)

def extract_entities_from_dataframe(dataframe, entity_type):
    extracted_entities = []

    for idx, row in dataframe.iterrows():
        sentence = row['Sentence']
        tag_sequence = row['Tag']
        entity = ''
        inside_entity = False
        tags_split = tag_sequence.split()

        for word, tag in zip(re.findall(r'\S+', sentence), tags_split):
            if entity_type in tag:
                if 'B' in tag:
                    entity = word
                    inside_entity = True
                elif inside_entity:
                    entity += ' ' + word
            else:
                if inside_entity:
                    extracted_entities.append(entity)
                    entity = ''
                    inside_entity = False

        if inside_entity:
            extracted_entities.append(entity)

    return extracted_entities

In [ ]:
test_ner_df = pd.read_csv('ner_test_data.csv')

In [ ]:
# 테스트데이터 result의 모든 ORG 태그를 LOC로 바꾸는 코드
test_ner_df['Tag'] = test_ner_df['Tag'].str.replace('ORG-B', 'LOC')
test_ner_df['Tag'] = test_ner_df['Tag'].str.replace('ORG-I', 'LOC')
test_ner_df['Tag'] = test_ner_df['Tag'].str.replace('LOC-B', 'LOC')
test_ner_df['Tag'] = test_ner_df['Tag'].str.replace('LOC-I', 'LOC')

In [ ]:
train_data_sentence = [sent.split() for sent in result['Sentence'].values]
test_data_sentence = [sent.split() for sent in test_ner_df['Sentence'].values]
train_data_label = [tag.split() for tag in result['Tag'].values]
test_data_label = [tag.split() for tag in test_ner_df['Tag'].values]

In [ ]:
labels = [label.strip() for label in open('ner_label.txt', 'r', encoding='utf-8')]

In [ ]:
labels.remove('ORG-B')
labels.remove('ORG-I')
labels.remove('LOC-B')
labels.remove('LOC-I')
labels.append('LOC')

In [ ]:
tag_to_index = {tag: index for index, tag in enumerate(labels)}
index_to_tag = {index: tag for index, tag in enumerate(labels)}

In [ ]:
tag_size = len(tag_to_index)

#2. 전처리

In [ ]:
tokenizer = BertTokenizer.from_pretrained("klue/bert-base")

In [ ]:
def convert_examples_to_features(examples, labels, max_seq_len, tokenizer,
                                 pad_token_id_for_segment=0, pad_token_id_for_label=-100):
    cls_token = tokenizer.cls_token
    sep_token = tokenizer.sep_token
    pad_token_id = tokenizer.pad_token_id

    input_ids, attention_masks, token_type_ids, data_labels = [], [], [], []

    for example, label in tqdm(zip(examples, labels), total=len(examples)):
        tokens = []
        labels_ids = []
        for one_word, label_token in zip(example, label):
            subword_tokens = tokenizer.tokenize(one_word)
            tokens.extend(subword_tokens)
            labels_ids.extend([tag_to_index[label_token]]+ [pad_token_id_for_label] * (len(subword_tokens) - 1))

        special_tokens_count = 2
        if len(tokens) > max_seq_len - special_tokens_count:
            tokens = tokens[:(max_seq_len - special_tokens_count)]
            labels_ids = labels_ids[:(max_seq_len - special_tokens_count)]

        tokens += [sep_token]
        labels_ids += [pad_token_id_for_label]
        tokens = [cls_token] + tokens
        labels_ids = [pad_token_id_for_label] + labels_ids

        input_id = tokenizer.convert_tokens_to_ids(tokens)
        attention_mask = [1] * len(input_id)
        padding_count = max_seq_len - len(input_id)

        input_id = input_id + ([pad_token_id] * padding_count)
        attention_mask = attention_mask + ([0] * padding_count)
        token_type_id = [pad_token_id_for_segment] * max_seq_len
        label = labels_ids + ([pad_token_id_for_label] * padding_count)

        assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
        assert len(attention_mask) == max_seq_len, "Error with attention mask length {} vs {}".format(len(attention_mask), max_seq_len)
        assert len(token_type_id) == max_seq_len, "Error with token type length {} vs {}".format(len(token_type_id), max_seq_len)
        assert len(label) == max_seq_len, "Error with labels length {} vs {}".format(len(label), max_seq_len)

        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        data_labels.append(label)

    input_ids = np.array(input_ids, dtype=int)
    attention_masks = np.array(attention_masks, dtype=int)
    token_type_ids = np.array(token_type_ids, dtype=int)
    data_labels = np.asarray(data_labels, dtype=np.int32)

    return (input_ids, attention_masks, token_type_ids), data_labels

In [ ]:
X_train, y_train = convert_examples_to_features(train_data_sentence, train_data_label, max_seq_len=128, tokenizer=tokenizer)

100%|██████████| 148509/148509 [02:22<00:00, 1043.72it/s]


In [ ]:
X_test, y_test = convert_examples_to_features(test_data_sentence, test_data_label, max_seq_len=128, tokenizer=tokenizer)

100%|██████████| 9000/9000 [00:05<00:00, 1619.10it/s]


In [ ]:
class TFBertForTokenClassification(tf.keras.Model):
    def __init__(self, model_name, num_labels):
        super(TFBertForTokenClassification, self).__init__()
        self.bert = TFBertModel.from_pretrained(model_name, from_pt=True)
        self.classifier = tf.keras.layers.Dense(num_labels,kernel_initializer=tf.keras.
                                                initializers.TruncatedNormal(0.02),name='classifier')

    def call(self, inputs):
        input_ids, attention_mask, token_type_ids = inputs
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask,
                            token_type_ids=token_type_ids)
        # 전 체 시 퀀 스 에 대 해 서 분 류 해 야 하 므 로 outputs[0]임 에 주 의
        all_output = outputs[0]
        prediction = self.classifier(all_output)

        return prediction

In [ ]:
labels = tf.constant([[-100, 2, 1, -100]])
logits = tf.constant([[[0.8, 0.1, 0.1], [0.06, 0.04, 0.9], [0.75, 0.1, 0.15],[0.4, 0.5, 0.1]]])

In [ ]:
active_loss = tf.reshape(labels, (-1,)) != -100
print(active_loss)

tf.Tensor([False  True  True False], shape=(4,), dtype=bool)


In [ ]:
reduced_logits = tf.boolean_mask(tf.reshape(logits, (-1, shape_list(logits)[2])),active_loss)
print(reduced_logits)

tf.Tensor(
[[0.06 0.04 0.9 ]
 [0.75 0.1  0.15]], shape=(2, 3), dtype=float32)


In [ ]:
labels = tf.boolean_mask(tf.reshape(labels, (-1,)), active_loss)
print(labels)

tf.Tensor([2 1], shape=(2,), dtype=int32)


In [ ]:
def compute_loss(labels, logits):
    # 다 중 클 래 스 분 류 문 제 에 서 소 프 트 맥 스 함 수 미 사 용 시 from_logits=True로 설 정.
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction=tf.keras.losses.Reduction.NONE)

    # -100의 값 을 가 진 정 수 에 대 해 서 는 오 차 를 반 영 하 지 않 도 록 labels를 수 정.
    active_loss = tf.reshape(labels, (-1,)) != -100

    # activa_loss로 부 터 reduced_logits과 labels를 각 각 얻 는 다.
    reduced_logits = tf.boolean_mask(tf.reshape(logits, (-1, shape_list(logits)[2])), active_loss)
    labels = tf.boolean_mask(tf.reshape(labels, (-1,)), active_loss)

    return loss_fn(labels, reduced_logits)

#3. 모델링과 학습


In [ ]:
from transformers import TFBertForTokenClassification

In [ ]:
# TPU 작 동 을 위 한 코 드
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.experimental.TPUStrategy(resolver)

In [ ]:
with strategy.scope():
  model = TFBertForTokenClassification.from_pretrained("klue/bert-base", num_labels=tag_size, from_pt=True)
  optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
  model.compile(optimizer=optimizer, loss=model.hf_compute_loss)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForTokenClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForTokenClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForTokenClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForTokenClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
class F1score(tf.keras.callbacks.Callback):
    def __init__(self, X_test, y_test):
        self.X_test = X_test
        self.y_test = y_test

    def sequences_to_tags(self, label_ids, pred_ids):
      label_list = []
      pred_list = []

      for i in range(0, len(label_ids)):
        label_tag = []
        pred_tag = []

        for label_index, pred_index in zip(label_ids[i], pred_ids[i]):
          if label_index != -100:
            label_tag.append(index_to_tag[label_index])
            pred_tag.append(index_to_tag[pred_index])

        label_list.append(label_tag)
        pred_list.append(pred_tag)

      return label_list, pred_list

    def on_epoch_end(self, epoch, logs={}):
      y_predicted = self.model.predict(self.X_test)
      y_predicted = np.argmax(y_predicted.logits, axis = 2)

      label_list, pred_list = self.sequences_to_tags(self.y_test, y_predicted)

      score = f1_score(label_list, pred_list, suffix=True)
      print(' - f1: {:04.2f}'.format(score * 100))
      print(classification_report(label_list, pred_list, suffix=True))

In [ ]:
f1_score_report = F1score(X_test, y_test)

In [ ]:
model.fit(
    X_train, y_train, epochs=8, batch_size=64,
    callbacks = [f1_score_report]
)

Epoch 1/8
   6/2321 [..............................] - ETA: 3:51 - loss: 2.0220

282/282 [==============================] - 21s 52ms/step


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:167: UserWarning: LOC seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


 - f1: 88.84
              precision    recall  f1-score   support

         AFW       0.67      0.68      0.67       394
         ANM       0.78      0.83      0.81       701
         CVL       0.89      0.86      0.87      5758
         DAT       0.93      0.94      0.93      2521
         EVT       0.80      0.79      0.80      1094
         FLD       0.81      0.60      0.69       228
          LO       0.89      0.91      0.90      5456
         MAT       0.50      0.25      0.33        12
         NUM       0.93      0.93      0.93      5590
         PER       0.92      0.91      0.92      4426
         PLT       0.75      0.26      0.39        34
         TIM       0.83      0.93      0.88       314
         TRM       0.83      0.77      0.80      1964

   micro avg       0.89      0.88      0.89     28492
   macro avg       0.81      0.74      0.76     28492
weighted avg       0.89      0.88      0.89     28492

2321/2321 [==============================] - 396s 125ms/step - los

In [43]:
def convert_examples_to_features_for_prediction(examples, max_seq_len, tokenizer,
                                 pad_token_id_for_segment=0, pad_token_id_for_label=-100):
    cls_token = tokenizer.cls_token
    sep_token = tokenizer.sep_token
    pad_token_id = tokenizer.pad_token_id

    input_ids, attention_masks, token_type_ids, label_masks = [], [], [], []

    for example in tqdm(examples):
        tokens = []
        label_mask = []
        for one_word in example:
            subword_tokens = tokenizer.tokenize(one_word)
            tokens.extend(subword_tokens)
            label_mask.extend([0]+ [pad_token_id_for_label] * (len(subword_tokens) - 1))

        special_tokens_count = 2
        if len(tokens) > max_seq_len - special_tokens_count:
            tokens = tokens[:(max_seq_len - special_tokens_count)]
            label_mask = label_mask[:(max_seq_len - special_tokens_count)]

        tokens += [sep_token]
        label_mask += [pad_token_id_for_label]
        tokens = [cls_token] + tokens
        label_mask = [pad_token_id_for_label] + label_mask
        input_id = tokenizer.convert_tokens_to_ids(tokens)
        attention_mask = [1] * len(input_id)
        padding_count = max_seq_len - len(input_id)
        input_id = input_id + ([pad_token_id] * padding_count)
        attention_mask = attention_mask + ([0] * padding_count)
        token_type_id = [pad_token_id_for_segment] * max_seq_len
        label_mask = label_mask + ([pad_token_id_for_label] * padding_count)

        assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
        assert len(attention_mask) == max_seq_len, "Error with attention mask length {} vs {}".format(len(attention_mask), max_seq_len)
        assert len(token_type_id) == max_seq_len, "Error with token type length {} vs {}".format(len(token_type_id), max_seq_len)
        assert len(label_mask) == max_seq_len, "Error with labels length {} vs {}".format(len(label_mask), max_seq_len)

        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        label_masks.append(label_mask)

    input_ids = np.array(input_ids, dtype=int)
    attention_masks = np.array(attention_masks, dtype=int)
    token_type_ids = np.array(token_type_ids, dtype=int)
    label_masks = np.asarray(label_masks, dtype=np.int32)

    return (input_ids, attention_masks, token_type_ids), label_masks

In [ ]:
X_pred, label_masks = convert_examples_to_features_for_prediction(test_data_sentence[:5], max_seq_len=128, tokenizer=tokenizer)

100%|██████████| 5/5 [00:00<00:00, 553.89it/s]


In [ ]:
def ner_prediction(examples, max_seq_len, tokenizer):
  examples = [sent.split() for sent in examples]
  X_pred, label_masks = convert_examples_to_features_for_prediction(examples, max_seq_len=128, tokenizer=tokenizer)
  y_predicted = model.predict(X_pred)
  y_predicted = np.argmax(y_predicted.logits, axis = 2)

  pred_list = []
  result_list = []

  for i in range(0, len(label_masks)):
    pred_tag = []
    for label_index, pred_index in zip(label_masks[i], y_predicted[i]):
      if label_index != -100:
        pred_tag.append(index_to_tag[pred_index])

    pred_list.append(pred_tag)

  for example, pred in zip(examples, pred_list):
    one_sample_result = []
    for one_word, label_token in zip(example, pred):
      one_sample_result.append((one_word, label_token))
    result_list.append(one_sample_result)

  return result_list

In [45]:
while True:
    sen = input('문장을 입력하세요: ')

    if sen == '끝':
        break

    else:
        test_samples = [sen]
        result_list = ner_prediction(test_samples, max_seq_len=128, tokenizer=tokenizer)
        print('결과:')
        print(result_list)
        print()

문장을 입력하세요: 여기 신대방역 스타벅스인데요 사람이 쓰러져있어요 


100%|██████████| 1/1 [00:00<00:00, 813.95it/s]


1/1 [==============================] - 1s 626ms/step
결과:
[[('여기', 'O'), ('신대방역', 'LOC'), ('스타벅스인데요', 'LOC'), ('사람이', 'O'), ('쓰러져있어요', 'O')]]



KeyboardInterrupt: ignored